# Summary of Model Results
(without `ANNOTATION:NEITHER` or `ANNOTATION:CHILD` categories, reference group = `SPOUSE`)

In [1]:
from cleaning.caregivers.models import (
    load_data,
    run_logit,
    format_logit_results
)

from notebooks.models.ref import FORMULA_CONTROL, FORMULA_CONTROL_WITH_GOC
from tableone import TableOne

df = load_data()

In [2]:
df = df[
    (df["ANNOTATION"] != "NEITHER") &\
    (df["ANNOTATION"] != "CHILD")
]

In [3]:
responses = [
    "IDENTIFIED_CONV_GOC",
    "IDENTIFIED_CONV_LIM",
    "MORTALITY_3MO_FROM_HADM_ADMIT",
    "HOSPITAL_EXPIRE_FLAG"
]

formulas = {
    response:"{response} ~ {control_variables} + C(ANNOTATION, Treatment(reference='SPOUSE'))".format(control_variables=FORMULA_CONTROL, response=response)
    for response in responses
}

formulas_with_goc = {
    response:"{response} ~ {control_variables} + C(ANNOTATION, Treatment(reference='SPOUSE'))".format(control_variables=FORMULA_CONTROL_WITH_GOC, response=response)
    for response in responses
}

models = {
    response:run_logit(formulas[response], df, disp=False)
    for response in responses if response
}

models_with_goc = {
    response:run_logit(formulas_with_goc[response], df, disp=False)
    for response in responses if response != "IDENTIFIED_CONV_GOC"
}

/Users/annekwok/Desktop/caregivers/venv-caregivers/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


## GOC Conversation

The adjusted `IDENTIFIED_CONV_GOC` model doesn't converge with `FORMULA_CONTROL` with this data:

In [4]:
format_logit_results(models["IDENTIFIED_CONV_GOC"])

/Users/annekwok/Desktop/caregivers/cleaning/caregivers/models.py:61: RuntimeWarning: overflow encountered in exp
  ci = np.exp(model.conf_int())


,odds ratio,"95% CI, lower","95% CI, upper",p-value
Intercept,1.336300e+00,0.4420,4.0406,0.6076
SEX[T.F],1.004500e+00,0.6469,1.5599,0.9840
MARITAL_STATUS[T.NOT MARRIED],8.339000e-01,0.4808,1.4464,0.5180
MARITAL_STATUS[T.UNKNOWN/NOT SPECIFIED],2.481259e+09,0.0000,inf,0.9994
ETHNICITY[T.OTHER],1.063900e+00,0.5719,1.9791,0.8450
ETHNICITY[T.UNKNOWN/NOT SPECIFIED],1.226100e+00,0.4351,3.4556,0.6998
ANNOTATION[T.BOTH],1.352100e+00,0.8346,2.1903,0.2204
ADMISSION_AGE,1.004200e+00,0.9875,1.0211,0.6262
ELIX_SCORE,9.749000e-01,0.8845,1.0745,0.6086
SOFA,1.103100e+00,1.0366,1.1738,0.0020


This is likely due to the fact that all the examples where `MARITAL_STATUS:UNKNOWN/NOT SPECIFIED` have `IDENTIFIED_CONV_GOC:1`:

In [5]:
TableOne(
    df,
    columns=["MARITAL_STATUS"],
    groupby="IDENTIFIED_CONV_GOC"
)

Grouped by IDENTIFIED_CONV_GOC                                   
                                                                   Missing     Overall          0           1
n                                                                                  506        121         385
MARITAL_STATUS, n (%) MARRIED                                            0  407 (80.4)  95 (78.5)  312 (81.0)
                      NOT MARRIED                                            89 (17.6)  26 (21.5)   63 (16.4)
                      UNKNOWN/NOT SPECIFIED                                   10 (2.0)               10 (2.6)

If we re-run the model with these ten examples dropped, we can confirm the above values are actually "correct":

In [6]:
formula = "{response} ~ {control_variables} + C(ANNOTATION, Treatment(reference='SPOUSE'))".format(
    response="IDENTIFIED_CONV_GOC",
    control_variables=FORMULA_CONTROL
)

model = run_logit(formula, df[df["MARITAL_STATUS"] != "UNKNOWN/NOT SPECIFIED"])

format_logit_results(model)

Optimization terminated successfully.
         Current function value: 0.539045
         Iterations 6


,odds ratio,"95% CI, lower","95% CI, upper",p-value
Intercept,1.3363,0.4420,4.0406,0.6076
SEX[T.F],1.0045,0.6469,1.5599,0.9840
MARITAL_STATUS[T.NOT MARRIED],0.8339,0.4808,1.4464,0.5180
ETHNICITY[T.OTHER],1.0639,0.5719,1.9791,0.8450
ETHNICITY[T.UNKNOWN/NOT SPECIFIED],1.2261,0.4351,3.4556,0.6998
ANNOTATION[T.BOTH],1.3521,0.8346,2.1903,0.2204
ADMISSION_AGE,1.0042,0.9875,1.0211,0.6262
ELIX_SCORE,0.9749,0.8845,1.0745,0.6086
SOFA,1.1031,1.0366,1.1738,0.0020


### Unadjusted

In [7]:
formula = "IDENTIFIED_CONV_GOC ~ C(ANNOTATION, Treatment(reference='SPOUSE'))"

model = run_logit(formula, df)

format_logit_results(model)

Optimization terminated successfully.
         Current function value: 0.545222
         Iterations 5


,odds ratio,"95% CI, lower","95% CI, upper",p-value
Intercept,2.6627,2.0689,3.4268,0.0000
ANNOTATION[T.BOTH],1.6209,1.0504,2.5012,0.0291


### Adjusted for GOC

N/A

## LIM Conversation

In [8]:
format_logit_results(models["IDENTIFIED_CONV_LIM"])

,odds ratio,"95% CI, lower","95% CI, upper",p-value
Intercept,0.0135,0.0036,0.0507,0.0000
SEX[T.F],0.8027,0.4984,1.2928,0.3661
MARITAL_STATUS[T.NOT MARRIED],1.2042,0.6496,2.2325,0.5551
MARITAL_STATUS[T.UNKNOWN/NOT SPECIFIED],1.0102,0.1877,5.4359,0.9906
ETHNICITY[T.OTHER],1.3803,0.7508,2.5377,0.2996
ETHNICITY[T.UNKNOWN/NOT SPECIFIED],0.4664,0.1425,1.5268,0.2075
ANNOTATION[T.BOTH],1.7555,1.0927,2.8205,0.0200
ADMISSION_AGE,1.0304,1.0115,1.0497,0.0015
ELIX_SCORE,0.9637,0.8715,1.0657,0.4718
SOFA,1.1858,1.1177,1.2582,0.0000


### Unadjusted

In [9]:
formula = "IDENTIFIED_CONV_LIM ~ C(ANNOTATION, Treatment(reference='SPOUSE'))"

model = run_logit(formula, df)

format_logit_results(model)

Optimization terminated successfully.
         Current function value: 0.536060
         Iterations 5


,odds ratio,"95% CI, lower","95% CI, upper",p-value
Intercept,0.2063,0.1531,0.2781,0.0
ANNOTATION[T.BOTH],2.7424,1.8125,4.1495,0.0


### Adjusted for GOC

In [10]:
format_logit_results(models_with_goc["IDENTIFIED_CONV_LIM"])

,odds ratio,"95% CI, lower","95% CI, upper",p-value
Intercept,0.0022,0.0004,0.0112,0.0000
SEX[T.F],0.7855,0.4796,1.2865,0.3375
MARITAL_STATUS[T.NOT MARRIED],1.1885,0.6241,2.2636,0.5993
MARITAL_STATUS[T.UNKNOWN/NOT SPECIFIED],0.7383,0.1374,3.9661,0.7236
ETHNICITY[T.OTHER],1.2976,0.6894,2.4423,0.4195
ETHNICITY[T.UNKNOWN/NOT SPECIFIED],0.4844,0.1427,1.6447,0.2452
ANNOTATION[T.BOTH],1.6547,1.0066,2.7201,0.0471
ADMISSION_AGE,1.0288,1.0094,1.0486,0.0035
ELIX_SCORE,0.9631,0.8670,1.0698,0.4827
SOFA,1.1675,1.0985,1.2409,0.0000


## Mortality (3-month)

In [11]:
format_logit_results(models["MORTALITY_3MO_FROM_HADM_ADMIT"])

,odds ratio,"95% CI, lower","95% CI, upper",p-value
Intercept,0.0639,0.0210,0.1942,0.0000
SEX[T.F],0.6806,0.4481,1.0335,0.0710
MARITAL_STATUS[T.NOT MARRIED],0.5682,0.3190,1.0120,0.0549
MARITAL_STATUS[T.UNKNOWN/NOT SPECIFIED],1.1329,0.2770,4.6340,0.8621
ETHNICITY[T.OTHER],0.9394,0.5312,1.6611,0.8297
ETHNICITY[T.UNKNOWN/NOT SPECIFIED],0.7483,0.2900,1.9304,0.5487
ANNOTATION[T.BOTH],1.5032,0.9840,2.2964,0.0594
ADMISSION_AGE,1.0198,1.0035,1.0363,0.0170
ELIX_SCORE,1.0028,0.9184,1.0950,0.9498
SOFA,1.1699,1.1073,1.2360,0.0000


### Unadjusted

In [12]:
formula = "MORTALITY_3MO_FROM_HADM_ADMIT ~ C(ANNOTATION, Treatment(reference='SPOUSE'))"

model = run_logit(formula, df)

format_logit_results(model)

Optimization terminated successfully.
         Current function value: 0.644866
         Iterations 5


,odds ratio,"95% CI, lower","95% CI, upper",p-value
Intercept,0.4340,0.3398,0.5543,0.0
ANNOTATION[T.BOTH],2.1714,1.5017,3.1398,0.0


### Adjusted for GOC

In [13]:
format_logit_results(models_with_goc["MORTALITY_3MO_FROM_HADM_ADMIT"])

,odds ratio,"95% CI, lower","95% CI, upper",p-value
Intercept,0.0408,0.0126,0.1319,0.0000
SEX[T.F],0.6808,0.4472,1.0365,0.0730
MARITAL_STATUS[T.NOT MARRIED],0.5718,0.3195,1.0234,0.0598
MARITAL_STATUS[T.UNKNOWN/NOT SPECIFIED],0.9769,0.2391,3.9920,0.9741
ETHNICITY[T.OTHER],0.9116,0.5128,1.6204,0.7524
ETHNICITY[T.UNKNOWN/NOT SPECIFIED],0.7648,0.2975,1.9665,0.5779
ANNOTATION[T.BOTH],1.4602,0.9518,2.2401,0.0830
ADMISSION_AGE,1.0196,1.0032,1.0362,0.0190
ELIX_SCORE,1.0037,0.9183,1.0970,0.9356
SOFA,1.1607,1.0982,1.2268,0.0000


## Mortality (in-hospital)

In [14]:
format_logit_results(models["HOSPITAL_EXPIRE_FLAG"])

,odds ratio,"95% CI, lower","95% CI, upper",p-value
Intercept,0.0466,0.0137,0.1591,0.0000
SEX[T.F],0.6275,0.3938,1.0000,0.0500
MARITAL_STATUS[T.NOT MARRIED],0.6354,0.3363,1.2006,0.1624
MARITAL_STATUS[T.UNKNOWN/NOT SPECIFIED],1.4780,0.3464,6.3052,0.5976
ETHNICITY[T.OTHER],1.0329,0.5583,1.9111,0.9179
ETHNICITY[T.UNKNOWN/NOT SPECIFIED],1.1792,0.4328,3.2131,0.7472
ANNOTATION[T.BOTH],1.9526,1.2257,3.1106,0.0049
ADMISSION_AGE,1.0085,0.9908,1.0265,0.3494
ELIX_SCORE,1.0055,0.9135,1.1067,0.9114
SOFA,1.2327,1.1602,1.3097,0.0000


### Unadjusted

In [15]:
formula = "HOSPITAL_EXPIRE_FLAG ~ C(ANNOTATION, Treatment(reference='SPOUSE'))"

model = run_logit(formula, df)

format_logit_results(model)

Optimization terminated successfully.
         Current function value: 0.575399
         Iterations 5


,odds ratio,"95% CI, lower","95% CI, upper",p-value
Intercept,0.2614,0.1981,0.3450,0.0
ANNOTATION[T.BOTH],2.5608,1.7250,3.8015,0.0


### Adjusted for GOC

In [16]:
format_logit_results(models_with_goc["HOSPITAL_EXPIRE_FLAG"])

,odds ratio,"95% CI, lower","95% CI, upper",p-value
Intercept,0.0211,0.0056,0.0801,0.0000
SEX[T.F],0.6245,0.3893,1.0017,0.0508
MARITAL_STATUS[T.NOT MARRIED],0.6382,0.3346,1.2174,0.1729
MARITAL_STATUS[T.UNKNOWN/NOT SPECIFIED],1.1714,0.2745,4.9979,0.8308
ETHNICITY[T.OTHER],0.9802,0.5248,1.8307,0.9499
ETHNICITY[T.UNKNOWN/NOT SPECIFIED],1.2489,0.4578,3.4072,0.6643
ANNOTATION[T.BOTH],1.8985,1.1805,3.0533,0.0082
ADMISSION_AGE,1.0075,0.9896,1.0257,0.4166
ELIX_SCORE,1.0078,0.9141,1.1111,0.8758
SOFA,1.2219,1.1492,1.2992,0.0000
